In [2]:
# This code is adapted from [How I Created an AI Version of Myself] by Keith McNulty
# Original repository: https://github.com/keithmcnulty/rag_gemma_regression_book/blob/main/rag_gemma_experiment_open.ipynb
# Modifications by Aleksand Botvin
# I have made the following modifications:
#  - Read PDF files
#  - Using serverless LLM
#  - Creating a Telegram bot as an interface

In [4]:
# Импортируем библиотеку для работы с ChromaDB
import chromadb
from chromadb.utils import embedding_functions
from chromadb.utils.batch_utils import create_batches

# Импортируем клиент OpenAI для работы с API OpenRouter
from openai import OpenAI

In [6]:
# Установка необходимых пакетов:
# pip install python-telegram-bot

# Подключаем пакеты для работы с Telegram
import logging
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters

# Для работы с регулярными выражениями
import re

In [8]:
# Импорт и применение nest_asyncio для обеспечения корректной работы асинхронного кода в Jupyter или других окружениях
import nest_asyncio
nest_asyncio.apply()

In [10]:
# Указываем путь к локальной базе данных ChromaDB
CHROMA_DATA_PATH = "chroma_data_bayesium/"

# Используем предобученную модель для получения эмбеддингов
EMBED_MODEL = "all-MiniLM-L6-v2"

# Название коллекции, в которой хранятся документы
COLLECTION_NAME = "bayesium"

# Инициализируем клиента ChromaDB для работы с персистентным хранилищем
chromadb_client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

# Переинициализируем функцию эмбеддинга с использованием выбранной модели
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL
)

# Получаем уже созданную коллекцию с книгами по байесовской статистике
collection = chromadb_client.get_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_func  
)

In [12]:
# Создаём клиента OpenAI для доступа к API OpenRouter 
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="<ВАШ API KEY>",
)

In [14]:
def ask_question(question: str, n_docs: int = 3) -> str:
    """
    Функция для обработки вопроса.
    
    Аргументы:
      question (str): Текст вопроса.
      n_docs (int): Количество документов, извлекаемых из базы для формирования ответа.
      
    Возвращает:
      str: Сгенерированный ответ LLM.
    """
    
    # Получаем коллекцию из базы ChromaDB
    collection = chromadb_client.get_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_func  
    )
    
    # Выполняем поиск по схожести: получаем n_docs наиболее релевантных документов
    results = collection.query(
        query_texts=[question],
        n_results=n_docs
    )
    
    # Формируем контекст с текстом документа и соответствующими метаданными (глава, источник)
    context_with_metadata = ""
    referenced_chapters = set()  # Множество для хранения уникальных названий глав
    referenced_sources = set()   # Множество для хранения уникальных источников

    # Проходим по найденным документам
    for i, document in enumerate(results['documents'][0]):
        # Получаем метаданные для текущего документа
        metadata = results['metadatas'][0][i]  
        chapter = metadata.get('chapter', 'Unknown Chapter')  
        referenced_chapters.add(chapter) 
        source = metadata.get('source') 
        referenced_sources.add(source)
        
        # Формируем строку с информацией о документе
        context_with_metadata += f"Document {i+1} (Chapter: {chapter}):\n"
        context_with_metadata += f"{document}\n"
        context_with_metadata += "-" * 50 + "\n"

    # Объединяем названия глав и источники в строку для дальнейшего упоминания в ответе
    chapters_list = ", ".join(referenced_chapters)
    sources = ", ".join(referenced_sources)

    # Формируем окончательный запрос для LLM, включающий вопрос и контекст
    prompt = f"""
    You are an expert on statistics and its applications to analytics.
    Here is a question: {question}
    Answer it strictly based on the following information:
    {context_with_metadata}
    When providing the answer, reference the chapters used by stating "This information is from {chapters_list} by {sources}".
    If the context does not provide enough information, say "I cannot answer this question based on the provided context."
    """

    # Оборачиваем запрос в формате сообщений для модели
    messages = [
        {"role": "user", "content": prompt}
    ]

    # Отправляем запрос на генерацию ответа LLM с помощью OpenRouter
    completion = client.chat.completions.create(
        model="meta-llama/llama-3.1-70b-instruct:free", 
        messages=messages,
        max_tokens=1000,
        temperature=0.25 
    )

    # Возвращаем сгенерированный ответ
    return completion.choices[0].message.content

In [16]:
# Разрешённый ID пользователя, которому разрешено использовать бота
ALLOWED_USER_ID = 164935376

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """
    Обрабатывает команду /start:
      - Проверяет, является ли пользователь авторизованным.
      - Отправляет приветственное сообщение.
    """
    user_id = update.effective_user.id
    if user_id != ALLOWED_USER_ID:
        await update.message.reply_text("Извините, но вы не можете воспользоваться этим ботом.")
        return

    await update.message.reply_text(
        "Привет! Я Байезиум - бот по байесовской статистике. Задай мне вопрос и я отвечу на основе книг Ричарда МакЭлрита и Джона Крушке"
    )



In [18]:
def format_code_for_telegram(text: str, parse_mode: str = "HTML") -> str:
    """
    Форматирует текст для Telegram с использованием HTML:
      - Экранирует специальные HTML-символы.
      - Преобразует блоки кода, выделенные с помощью ```...```, в HTML-теги <pre><code>...</code></pre>.
    """
    if parse_mode == "HTML":
        # Экранирование символов &, <, >
        text = text.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")
        # Замена блоков кода на HTML-формат
        text = re.sub(r"```(\w+)?\n(.*?)```", r"<pre><code>\2</code></pre>", text, flags=re.DOTALL)
        return text
    else:
        return text 

In [20]:
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """
    Обрабатывает входящие сообщения:
      - Проверяет авторизацию пользователя.
      - Извлекает текст вопроса из сообщения.
      - Уведомляет пользователя о начале обработки.
      - Вызывает функцию ask_question для получения ответа.
      - Форматирует ответ и отправляет его обратно пользователю.
    """
    
    user_id = update.effective_user.id
    if user_id != ALLOWED_USER_ID:
        await update.message.reply_text("Извините, но вы не можете воспользоваться этим ботом.")
        return

    question = update.message.text
    print(f"DEBUG: Received message - {question}")
    logging.info(f"DEBUG: Received message - {question}")

    # Информируем пользователя о том, что запрос обрабатывается
    await update.message.reply_text("Осуществляется обработка вашего вопроса, пожалуйста, ожидайте...")

    try:
        # Выполнение функции ask_question в отдельном потоке, чтобы не блокировать event loop
        answer = await asyncio.to_thread(ask_question, question)
        print(f"DEBUG: Answer generated - {answer}")
        logging.info(f"DEBUG: Answer generated - {answer}")

        # Форматирование ответа для корректного отображения в Telegram
        formatted_answer = format_code_for_telegram(answer, parse_mode="HTML")
    except Exception as e:
        logging.error(f"Error in ask_question: {e}")
        formatted_answer = "Sorry, something went wrong while processing your request."

    # Отправка отформатированного ответа пользователю
    await update.message.reply_text(formatted_answer, parse_mode="HTML")

In [ ]:
async def main() -> None:
    """
    Основная функция:
      - Создает экземпляр приложения Telegram-бота.
      - Регистрирует обработчики команд и сообщений.
      - Запускает цикл поллинга для получения обновлений.
    """
    application = ApplicationBuilder().token("<ВАШ API KEY>").build()

    # Регистрация обработчика команды /start
    application.add_handler(CommandHandler("start", start))
    # Регистрация обработчика для всех остальных сообщений
    application.add_handler(MessageHandler(filters.ALL, handle_message))

    # Запуск бота в режиме поллинга
    await application.run_polling()

if __name__ == '__main__':
    # Настройка логирования для отслеживания событий и ошибок
    logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
    )

    import asyncio
    # Запуск основной функции с использованием asyncio
    asyncio.run(main())

2025-02-15 16:17:57,099 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot/getMe "HTTP/1.1 200 OK"
2025-02-15 16:17:57,189 - httpx - INFO - HTTP Request: POST https://api.telegram.org/deleteWebhook "HTTP/1.1 200 OK"
2025-02-15 16:17:57,190 - telegram.ext.Application - INFO - Application started
2025-02-15 16:18:07,736 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot/getUpdates "HTTP/1.1 200 OK"
